In [3]:
#tt

In [ ]:
##### 
## The first part, capture 300 images of Jeff (myself) using the laptop camera; 
import cv2
import os
import time

# Set the directory where the images will be saved
save_dir = "C:\\Users\\duj\\OneDrive - QIAGEN GmbH\\Sanofi_projects\\Pipeline_Testing_joblogs\\TriangleAI_Presentation\\TrainYoloWithImages"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Initialize the camera (use 0 for the default camera)
cap = cv2.VideoCapture(1)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Set the number of images to capture
num_images = 300
interval = 3  # seconds between captures

# Start capturing images
for i in range(num_images):
    ret, frame = cap.read()  # Capture frame
    if not ret:
        print("Error: Failed to capture image.")
        break
    
    # Save the image
    img_name = os.path.join(save_dir, f"image_{i+1:03d}.png")
    cv2.imwrite(img_name, frame)
    print(f"Saved {img_name}")
    
    # Wait for 3 seconds before capturing the next image
    time.sleep(interval)

# Release the camera
cap.release()
cv2.destroyAllWindows()

print(f"Captured {num_images} images.")


In [ ]:
##### 
## The 2nd part, run labelImg app to label the images
## 

In [ ]:
##### 
## The 3rd part, since labelImg ouput Pascal VOC XML format *.xml files as labels, we need to transfer those Pascal VOC to YOLO Format
## 

In [ ]:
import os
import xml.etree.ElementTree as ET

# Paths
xml_dir = "C:\\Users\\duj\\OneDrive - QIAGEN GmbH\\Sanofi_projects\\Pipeline_Testing_joblogs\\TriangleAI_Presentation\\TrainYoloWithImages\\labels"  # Directory with XML files from LabelImg
output_dir = "C:\\Users\\duj\\OneDrive - QIAGEN GmbH\\Sanofi_projects\\Pipeline_Testing_joblogs\\TriangleAI_Presentation\\TrainYoloWithImages\\labels"   # Output directory for YOLO format labels
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to convert Pascal VOC XML to YOLO format
def convert_to_yolo(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image dimensions
    size = root.find("size")
    img_width = int(size.find("width").text)
    img_height = int(size.find("height").text)

    # Prepare YOLO format output
    yolo_lines = []

    # Iterate over each object in the XML
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        class_id = 0  # Assuming one class (yourself), you can adjust this

        # Get bounding box information
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)

        # Convert to YOLO format (normalized values)
        x_center = (xmin + xmax) / 2.0 / img_width
        y_center = (ymin + ymax) / 2.0 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        # Create YOLO format line
        yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}\n")

    # Return the YOLO formatted label
    return yolo_lines

# Process each XML file
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(xml_dir, xml_file)
        yolo_labels = convert_to_yolo(xml_path)

        # Save YOLO labels to txt file
        txt_filename = os.path.splitext(xml_file)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)
        with open(txt_path, "w") as f:
            f.writelines(yolo_labels)

        print(f"Converted {xml_file} to {txt_filename}")


In [ ]:
#### 
## part 4th, run YOLO model training to get a recognition model for Jeff :) 
## 

In [4]:
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8n.pt")  # 'n' means 'nano' version, suitable for fast training

# Train the model on your custom dataset
model.train(
    data="C:\\Users\\duj\\OneDrive - QIAGEN GmbH\\Sanofi_projects\\Pipeline_Testing_joblogs\\TriangleAI_Presentation\\data.yaml",  # Path to the dataset YAML file
    epochs=50,                            # Number of training epochs
    imgsz=640,                            # Image size for training
    batch=16,                             # Batch size
    name="jeff_yolo_self_recognition"     # Project name
) 

New https://pypi.org/project/ultralytics/8.3.3 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.87  Python-3.9.19 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\duj\OneDrive - QIAGEN GmbH\Sanofi_projects\Pipeline_Testing_joblogs\TriangleAI_Presentation\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=jeff_yolo_self_recognition5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fals

train: Scanning C:\Users\duj\OneDrive - QIAGEN GmbH\Sanofi_projects\Pipeline_Testing_joblogs\TriangleAI_Presentation\Tr
val: Scanning C:\Users\duj\OneDrive - QIAGEN GmbH\Sanofi_projects\Pipeline_Testing_joblogs\TriangleAI_Presentation\ValY


Plotting labels to runs\detect\jeff_yolo_self_recognition5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\jeff_yolo_self_recognition5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G     0.9543      2.072      1.369         44        640: 100%|██████████| 15/15 [01:43<00:00,  6.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<0

                   all         60         61    0.00339          1      0.982      0.709


...

                   all         60         61      0.999      0.984      0.985      0.864



50 epochs completed in 1.422 hours.
Optimizer stripped from runs\detect\jeff_yolo_self_recognition5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\jeff_yolo_self_recognition5\weights\best.pt, 6.2MB

Validating runs\detect\jeff_yolo_self_recognition5\weights\best.pt...
Ultralytics YOLOv8.2.87  Python-3.9.19 torch-2.3.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<0


                   all         60         61      0.999      0.984      0.986      0.868
Speed: 1.6ms preprocess, 103.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\jeff_yolo_self_recognition5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002575D00F160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [6]:
import os
os.getcwd()

'C:\\Users\\duj'

In [ ]:
## give the trained model a try :) 

In [7]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model_jeff = YOLO("C:\\Users\\duj\\OneDrive - QIAGEN GmbH\\Sanofi_projects\\Pipeline_Testing_joblogs\\TriangleAI_Presentation\\runs\\detect\\jeff_yolo_self_recognition4\\weights\\best.pt")
model_jeff = YOLO("C:\\Users\\duj\\runs\\detect\\jeff_yolo_self_recognition5\\weights\\best.pt")

# Open the webcam
cap = cv2.VideoCapture(1)

# Define a function to display the label
def draw_label(image, text, pos, color=(0, 255, 0), font=cv2.FONT_HERSHEY_SIMPLEX, scale=0.6, thickness=2):
    cv2.putText(image, text, pos, font, scale, color, thickness, cv2.LINE_AA)

# Loop to capture frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model_jeff.predict(source=frame)

    # Loop through the detected results
    for result in results:
        # Loop through each detection (each bounding box)
        for box in result.boxes.xyxy:  # Extract the bounding box coordinates
            x1, y1, x2, y2 = map(int, box)  # Convert to integer coordinates
            # Draw rectangle around the person
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle

            # Draw label "somebody" next to the bounding box
            draw_label(frame, "Jeff.G", (x1, y1 - 10))

    # Display the frame
    cv2.imshow("YOLOv8 - Self Recognition", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 Jeff.G, 102.8ms
Speed: 4.9ms preprocess, 102.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 120.8ms
Speed: 2.5ms preprocess, 120.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 94.0ms
Speed: 7.5ms preprocess, 94.0ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 88.9ms
Speed: 2.7ms preprocess, 88.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 103.7ms
Speed: 3.2ms preprocess, 103.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 95.3ms
Speed: 2.4ms preprocess, 95.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 111.0ms
Speed: 2.2ms preprocess, 111.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Jeff.G, 101.0ms
Speed: 2.9ms preprocess, 101.0ms inference, 1.2ms postprocess per image at shape